# Parameterizing Skills with the Solution Building Library

This example notebook demonstrates how to instantiate and parameterize skills with the Intrinsic Solution Building Library.

We pay special attention on how to do this in practice with the tools provided by Jupyter in VS Code.

<div class="alert alert-info">

**Important**

This notebook requires a running Flowstate solution to connect to. To start a solution:

1. Navigate to [portal.intrinsic.ai](https://portal.intrinsic.ai/) and sign in
   using your registered Flowstate account.

1. Do **one** of the following:
    - Create a new solution:
        1. Click "Create new solution" and choose "From an example".
        1. Select `building_block:building_block_module2`
        1. Click "Create".
    - Or open an existing solution that was created from the `building_block:building_block_module2` example:
        1. Hover over the solution in the list.
        1. Click "Open solution" or "Start solution".

1. Recommended: Keep the browser tab with the Flowstate solution editor open to watch the effect of notebook actions such as running a skill. You can simultaneously interact with the solution through the web UI and the notebook.

</div>

## Connect to solution

Let's start with the typical preamble:

- Import the relevant modules.
- Connect to the deployed solution.
- Define some shortcut variables for convenience.

In [ ]:
import math

from intrinsic.math.python import data_types
from intrinsic.solutions import deployments
from intrinsic.solutions import behavior_tree as bt

solution = deployments.connect_to_selected_solution()

executive = solution.executive
resources = solution.resources
skills = solution.skills
world = solution.world

Create an additional frame in the belief world that will be used by some examples:

In [ ]:
if 'pregrasp' not in world.building_block0.frame_names:
    world.create_frame(
        'pregrasp',
        world.building_block0,
        data_types.Pose3(
            rotation=data_types.Rotation3.from_euler_angles(rpy_degrees=[180, 0, 90]),
            translation=[0,0,0.05]))


At this point we recommend saving the solution in the Flowstate solution editor UI so that the created frame will persist across world resets.

## Creating your first skill instance

We use the `enable_gripper` skill as an example for creating our very first skill instance step-by-step.

If you haven't done so yet, try to find the skill via auto-completion. I.e., type `skills.` and press <kbd>Ctrl</kbd> + <kbd>Space</kbd> if the dropdown with suggestions does not show up automatically.

In [ ]:
# Type `skills.` and choose `enable_gripper`


Alternatively, you can also list the available skills in the solution with `dir()`:

In [ ]:
dir(skills)

The printed list should contain `enable_gripper`, indicating that `skills.enable_gripper` exists. `skills.enable_gripper` is a Python class that helps you to instantiate the skill with the appropriate parameters. The class gets dynamically generated by the Solution Building Library so that it exactly matches the skills version that is currently installed in your solution.

You get information about a skill helper class by using Python's `help()` method. It'll print information about the class, including the signature of its init method:

In [ ]:
help(skills.enable_gripper)

<div class="alert alert-info">

**Tip**

An alternative to calling the `help()` function manually in a notebook cell is to use the **Jupyter PowerToys** extension:

1. Click on the **Jupyter** icon <img src="https://raw.githubusercontent.com/microsoft/vscode-codicons/6ceb091d5c40da3e5836e3d80b08d3f74efc4cbf/src/icons/notebook.svg" width="25"> in the activity bar (to the far left side of the VS Code window).
1. Place your cursor, e.g., on `skills.enable_gripper` in the cell below or select any other code in a code cell.
1. See the `help` output shown live under **CONTEXTUAL HELP** in the sidebar (to the left side of the VS Code window).

You can also check out the [live animation of the contextual help feature](https://github.com/microsoft/vscode-jupyter-powertoys?tab=readme-ov-file#contextual-help) on the Jupyter PowerToys website.

</div>

The help output will show you what parameters the skill expects. In the case of the `enable_gripper` skill the printed signature is:

```
Skill_enable_gripper(
    *,
    clear_faults: bool,
    gripper: intrinsic.solutions.providers.ResourceHandle = ResourceHandle.create(...))
)
```

This means the only required parameter is a boolean and you can instantiate the skill as follows:

In [ ]:
enable_gripper_skill = skills.enable_gripper(clear_faults=True)

Note that for inclusion in a behavior tree, you would typically wrap the skill instance in a task node:

In [ ]:
enable_gripper = bt.Task(action=enable_gripper_skill, name='Enable gripper')

The skill also has a second parameter called `gripper` which is of `type intrinsic.solutions.providers.ResourceHandle`. It specifies which gripper in the solution to enable. In this simple case we don't have to explicitly specify this resource parameter because there is only one gripper resource in the solution. In other cases we have to specify resources explicitly which is detailed in the next section.

## Resources

`solution.Resources` is a lightweight wrapper that allows access to all the resource information that has been configured when authoring the solution. Resources can be passed to skills as a special type of parameter, e.g., when we want to specify which robot to move or which camera to use. You can get a list containing the names of all registered resources:


In [ ]:
dir(resources)


Each resource consists of a name and a list of capabilities. When providing a resource as a parameter to a skill, the skill's required capabilities get checked against this list of capabilities. A resource is considered compatible if the skill's required capabilities are a subset of the provided capabilities.

For example, in the building block solution a gripper resource with the name `picobot_gripper` is present.
If you print it, you can see the corresponding resource handle which is defined by its name `picobot_gripper` and its list of capabilities (=`types`).

In [ ]:
resources.picobot_gripper

You can also check which resources are compatible with a particular skill. E.g., you can list the resources which can be passed to the `gripper` parameter of the `enable_gripper` skill:

In [ ]:
dir(skills.enable_gripper.compatible_resources.gripper)

This means you can use `resources.picobot_gripper` for the `gripper` parameter of the `enable_gripper` skill. If there is only one resource in a solution that matches a skills requirements then the Solution Building Library will automatically determine a default parameter for the skill. This is why, in our simple solution, the `enable_gripper` skill can be instantiated equivalently in the following two ways:

In [ ]:
# Use default gripper (only possible if there is only one compatible gripper in the solution).
enable_gripper_skill = skills.enable_gripper(clear_faults=True)

# Explicitly specify which gripper to use.
enable_gripper_skill = skills.enable_gripper(
        clear_faults=True,
        gripper=resources.picobot_gripper)

## Nested skill parameters

The suction gripper in the building blocks solution can be controlled with the `control_suction_gripper` skill. Through `help(skills.control_suction_gripper)` you can get the skills signature:

```
Skill_control_suction_gripper(
    *,
    grasp: intrinsic.solutions.skills.control_suction_gripper.GraspRequest,
    release: intrinsic.solutions.skills.control_suction_gripper.ReleaseRequest,
    blow_off: intrinsic.solutions.skills.control_suction_gripper.BlowOffRequest,
    suction_gripper: intrinsic.solutions.providers.ResourceHandle = ResourceHandle.create(...))
)
```

You can see that the skill requires the three parameters `grasp`, `release` and `blow_off` with the types `intrinsic.solutions.skills.control_suction_gripper.GraspRequest/ReleaseRequest/BlowOffRequest`. Generally, skill parameters are defined by skill authors as [Protocol Buffer (proto)](https://protobuf.dev/) messages and the signature of a skill's helper class in Python is derived from the skill's parameter proto. The skill's parameter proto can contain [nested messages](https://protobuf.dev/programming-guides/proto3/#nested) in which case the Solution Building Library will automatically create helper classes for each type of nested message. In the case of `control_suction_gripper`, the following helper classes are generated:

- `skills.control_suction_gripper.GraspRequest`
- `skills.control_suction_gripper.ReleaseRequest`
- `skills.control_suction_gripper.BlowOffRequest`

Again, you can inspect these helper classes using `help()`, e.g.:

In [ ]:
help(skills.control_suction_gripper.GraspRequest)

All of these three helper classes (currently) take no parameters. That means, based on the skills Python signature, you'd expect that the following works:

In [ ]:
try:
    # Will raise an exception!
    skills.control_suction_gripper(
        grasp=skills.control_suction_gripper.GraspRequest(),
        release=skills.control_suction_gripper.ReleaseRequest(),
        blow_off=skills.control_suction_gripper.BlowOffRequest())
except Exception as e:
    # Print error message but do not fail the cell.
    print(repr(e))
    print(repr(e.__cause__))

However, the above will raise an exception that the parameters `grasp` and `release` cannot be passed at the same time. The Solution Building Library cannot map all protobuf features to Python perfectly. In this case, the skill's parameter proto contains a [oneof](https://protobuf.dev/programming-guides/proto3/#oneof) definition which cannot be indicated by the skills Python signature. Instead you get the observed runtime error.

The solution here is to only pass one of {`grasp`, `release` and `blow_off`} at a time which makes sense because we either want to "grasp", to "release" or to "blow off". Having this in mind, we can, e.g., create two working instances of the `control_suction_gripper` skill, one for grasping and one for releasing:

In [ ]:
grasp = skills.control_suction_gripper(
    grasp=skills.control_suction_gripper.GraspRequest())

release = skills.control_suction_gripper(
    release=skills.control_suction_gripper.ReleaseRequest())

## Motion skills

Various common robot motions can all be expressed and executed with a single skill: the `move_robot` skill. In this section we'll demonstrate how to use this skill.

### Motion targets

Motion targets can be defined in terms of Cartesian constraints or a robot joint configuration. The two most common motion target definitions are the Cartesian `PoseEquality` constraint and the `JointConfiguration`.

A `JointConfiguration` is a list of joint angles and can be obtained in various ways:

In [ ]:
# Joint configuration from plain values
joint_target_free = skills.move_robot.JointVec(
    joints=[
        math.radians(-90),
        math.radians(-90),
        math.radians(-90),
        math.radians(-90),
        math.radians(90),
        math.radians(90)])

# Named joint configuration stored in the world
joint_target_global_config = world.robot.joint_configurations.view_pose_left

# Joint configuration from the current robot position in the belief world,
# useful, e.g., after jogging the robot in the frontend
joint_target_world = skills.move_robot.JointVec(joints=world.robot.joint_positions)

These joint configurations can be used to create a `move_robot` skill instance as follows:

In [ ]:
# Disable collision checking.
collisions_disabled = skills.move_robot.CollisionSettings(disable_collision_checking=True)

# Define the single motion target as joint configuration in a motion_segment and
# disable collision checking for this segment.
move_home_unsafe = skills.move_robot(
    motion_segments=[
        skills.move_robot.MotionSegment(
            joint_position=joint_target_free,
            collision_settings=collisions_disabled)])

executive.run(move_home_unsafe)

Next to `JointMotionTarget`s there are also Cartesian motion target constraints. The most common constraint used to define a motion target is `PoseEquality` which defines the Cartesian pose of a frame attached to the robot (i.e., the moving frame). Besides the moving frame a target frame needs to be defined that defines the target pose of the moving frame.
Optionally, you can define an `target_frame_offset` between these two frames. The target offset defines the target position of the moving frame relative to the target frame.

To create a `PoseEquality` constraint which aligns the gripper tool frame with the pregrasp frame above the building block use:

In [ ]:
block_pregrasp = skills.move_robot.PoseEquality(
    moving_frame=world.picobot_gripper.tool_frame,
    target_frame=world.building_block0.pregrasp)

The motion can then get generated by creating a segment with a single cartesian motion target:

In [ ]:
move = skills.move_robot(
    motion_segments=[
        skills.move_robot.MotionSegment(cartesian_pose=block_pregrasp)])

executive.run(move)

The `PoseEquality` constraint also gives you the ability to configure an offset between the moving frame and the target frame.
This will move to 9cm above the center of the building block:

In [ ]:
above_block = skills.move_robot.PoseEquality(
    moving_frame=world.picobot_gripper.tool_frame,
    target_frame=world.building_block0,
    target_frame_offset=data_types.Pose3(
        rotation=data_types.Rotation3.from_euler_angles(rpy_degrees=[180, 0, 90]),
        translation=[0,0,0.09]))

move = skills.move_robot(
    motion_segments=[
        skills.move_robot.MotionSegment(cartesian_pose=above_block)])

executive.run(move)

To define a Cartesian motion relative to the current pose of a moving frame, set target frame equal to moving frame and define the relative motion using the target offset.
The following moton request will move the tool frame 9 cm in negative z-direction of the gripper tool frame.

In [ ]:
offset = data_types.Pose3(translation=[0,0,-0.09])
relative_pose = skills.move_robot.PoseEquality(
                        moving_frame=world.picobot_gripper.tool_frame,
                        target_frame=world.picobot_gripper.tool_frame,
                        target_frame_offset=offset)

move = skills.move_robot(
    motion_segments=[
        skills.move_robot.MotionSegment(cartesian_pose=relative_pose)])

executive.run(move)

### Linear Cartesian Motions

The previous motions find a (collision free) motion in joint configuration space and executes it. To plan and execute a Cartesian linear motion you can define a `linear_move` path constraint. The following motion moves down by 5 cm in z direction linear in Cartesian space:

In [ ]:
offset = data_types.Pose3(translation=[0,0,0.05])
relative_pose = skills.move_robot.PoseEquality(
                        moving_frame=world.picobot_gripper.tool_frame,
                        target_frame=world.picobot_gripper.tool_frame,
                        target_frame_offset=offset)

# Motion definition with single segment that defines a linear move to an orientation
move_down_linear = skills.move_robot(
    motion_segments=[
        skills.move_robot.MotionSegment(
            cartesian_pose=relative_pose,
            motion_type=skills.move_robot.MotionSegment.LINEAR)])

# Move robot to home pose first, then move to pregrasp
executive.run([move_home_unsafe, move_down_linear])

### Collision Settings
In the previous examples you have seen that sometimes the motion disables collision checking and sometimes collision checking is not set. By default, motion planning enables collision checking with the environment and attempts to find a collision free path to the motion target. If you want to disable collision checking you need to disable it for the respective motion segments:

In [ ]:
move_unsafe = skills.move_robot(
    motion_segments=[
        skills.move_robot.MotionSegment(
            joint_position=joint_target_global_config,
            collision_settings=collisions_disabled)])

executive.run(move_unsafe)

If collision settings is not set, a collision free trajectory is generated. By default, collision free means that no geometries are intersecting. While this is sufficient in theory when dealing with uncertainties in the environment it is recommended to set a minimum margin. The minimum margin enforces that the required motion has at least the distance defined in the margin to be considered collision free:

In [ ]:
# Define a collision margin of 1 cm
collision_margin = skills.move_robot.CollisionSettings(minimum_margin=0.01)

# Define a motion with one segment and a collision margin of 1 cm.
move_with_margin = skills.move_robot(
    motion_segments=[
        skills.move_robot.MotionSegment(
            joint_position=joint_target_free,
            collision_settings=collision_margin)])

executive.run(move_with_margin)

More complex collision settings can be designed using `CollisionRules`. `CollisionRules` allow to set object specific exclusion pairs and margins.

### More complex motions: Multi segment moves

To achieve more complex behavior, motion segments can be combined together to one motion by defining multi-segment moves.

The following move requests contains two motion segments. The first segments defines the motion that aligns the gripper tool frame with the pregrasp pose of the building block. The second motion segment defines the final target 10 cm in x direction relative to the pregrasp pose.

Note: When defining multi-segment motions, the robot will not stop at the waypoint. The robot will only come to an halt at the target of the final motion segment. The resulting motion will not pass through the exact waypoint. Instead, it passes the waypoints with a user configurable tightness.

In [ ]:
# Define planned move segment 1: Move to 15cm in -z-direction of pregrasp position
z_offset = -0.15
modified_pregrasp_position = skills.move_robot.PositionEquality(
    moving_frame=world.picobot_gripper.tool_frame,
    target_frame=world.building_block0.pregrasp,
    target_frame_offset=skills.move_robot.Point(z=z_offset))
motion_segment1 = skills.move_robot.MotionSegment(position_equality=modified_pregrasp_position)

# Define joint move segment 2: Move 10 cm in x-direction of the modified pregrasp. This is the final motion target.
relative_move_position = skills.move_robot.PositionEquality(
    moving_frame=world.picobot_gripper.tool_frame,
    target_frame=world.building_block0.pregrasp,
    target_frame_offset=skills.move_robot.Point(x=0.1, z=z_offset))
motion_segment2 = skills.move_robot.MotionSegment(position_equality=relative_move_position)

# Define a blending parameter that ensures that we pass through the pregrasp position (motion target segment 1)
# with an accuracy of 0.01 rad in joint configuration space
blending_parameter = skills.move_robot.BlendingParameters(
    joint_blending=skills.move_robot.JointBlendingParameters(
        desired_tightness_rad=0.01))

# Define move_robot skill that moves the end-effector to a 10 cm offset location (x-axis block) moving through the pregrasp position.
multi_segment_move = skills.move_robot(
    motion_segments=[motion_segment1, motion_segment2],
    curve_parameters=blending_parameter)

executive.run(multi_segment_move)

## Next steps

Take a look at the following example notebooks to learn:

- How to create behavior trees with control flow nodes such as [sequences](005_sequence.ipynb), [loops and branches](006_loop_and_branch.ipynb) or [retries](007_retry.ipynb).